In [ ]:
!pip install boto3
import boto3
import os
import requests
import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.4 MB/s eta 0:00:00


In [ ]:
dynamo_client  =  boto3.resource(service_name = 'dynamodb',region_name = 'us-east-1',
              aws_access_key_id = '************',
              aws_secret_access_key = '********************')

In [ ]:
dynamo_client.get_available_subresources()

['Table']

In [ ]:
data_table = dynamo_client.Table('TemperatureHumidity')
data_table.table_status

'ACTIVE'

In [ ]:
data_table.scan()['Items']

[{'humidity': Decimal('19.29999924'),
  'temperature': Decimal('1.100000024'),
  'timestamp': '1970-01-01 00:00:04'},
 {'humidity': Decimal('21'),
  'temperature': Decimal('1.700000048'),
  'timestamp': '1970-01-01 00:10:42'},
 {'humidity': Decimal('19.79999924'),
  'temperature': Decimal('1.600000024'),
  'timestamp': '1970-01-01 00:04:29'},
 {'humidity': Decimal('21.10000038'),
  'temperature': Decimal('1.700000048'),
  'timestamp': '1970-01-01 00:11:05'},
 {'humidity': Decimal('21'),
  'temperature': Decimal('1.700000048'),
  'timestamp': '1970-01-01 00:10:26'},
 {'humidity': Decimal('19.60000038'),
  'temperature': Decimal('1.5'),
  'timestamp': '1970-01-01 00:03:09'},
 {'humidity': Decimal('20.5'),
  'temperature': Decimal('1.600000024'),
  'timestamp': '1970-01-01 00:07:10'},
 {'humidity': Decimal('19'),
  'temperature': Decimal('1.399999976'),
  'timestamp': '1970-01-01 00:01:09'},
 {'humidity': Decimal('21.5'),
  'temperature': Decimal('1.600000024'),
  'timestamp': '1970-01-01

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
table = dynamo_client.Table('TemperatureHumidity')
response = table.scan(
    ExpressionAttributeNames={'#ts': 'timestamp'},
    ProjectionExpression='#ts, temperature, humidity'
)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from decimal import Decimal
# Create a Pandas DataFrame from the response
data_list = response['Items']
data_df = pd.DataFrame(data_list)

# Convert timestamp to datetime and set it as the index
data_df['timestamp'] = pd.to_datetime(data_df['timestamp'])
data_df.set_index('timestamp', inplace=True)

In [ ]:
data_df.dtypes

humidity       object
temperature    object
dtype: object

In [ ]:
data_df['humidity'] = data_df['humidity'].astype(float)
data_df['temperature'] = data_df['temperature'].astype(float)

In [ ]:
!pip install pmdarima

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(dataset):
  dftest = adfuller(dataset, autolag = 'AIC')
  print("1. ADF : ",dftest[0])
  print("2. P-Value : ", dftest[1])
  print("3. Num Of Lags : ", dftest[2])
  print("4. Num Of Observations Used For ADF Regression and Critical Values Calculation :", dftest[3])
  print("5. Critical Values :")
  for key, val in dftest[4].items():
      print("\t",key, ": ", val)

In [ ]:
adf_test(data_df['temperature'])

1. ADF :  -4.474921653649168
2. P-Value :  0.0002182077535611803
3. Num Of Lags :  18
4. Num Of Observations Used For ADF Regression and Critical Values Calculation : 841
5. Critical Values :
	 1% :  -3.4381494909387555
	 5% :  -2.864982795593248
	 10% :  -2.568603222439172


In [ ]:
adf_test(data_df['humidity'])

1. ADF :  -4.78217436171281
2. P-Value :  5.88209888070204e-05
3. Num Of Lags :  18
4. Num Of Observations Used For ADF Regression and Critical Values Calculation : 841
5. Critical Values :
	 1% :  -3.4381494909387555
	 5% :  -2.864982795593248
	 10% :  -2.568603222439172


In [ ]:
from pmdarima import auto_arima
# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
stepwise_fit = auto_arima(data_df['temperature'],
                          suppress_warnings=True)

stepwise_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  860
Model:               SARIMAX(5, 1, 3)   Log Likelihood               -1830.123
Date:                Thu, 10 Aug 2023   AIC                           3678.246
Time:                        12:44:39   BIC                           3721.048
Sample:                             0   HQIC                          3694.633
                                - 860                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -1.1156      0.237     -4.700      0.000      -1.581      -0.650
ar.L2         -0.3675      0.291     -1.265      0.206      -0.937       0.202
ar.L3          0.0537      0.170      0.316      0.752      -0.279       0.387
ar.L4         -0.0676      0.186     -0.363      0.717      -0.433       0.298
ar.L5         -0.1480      0.079     -1.867      0.062      -0.303       0.007
ma.L1          0.1084      0.218      0.496      0.620      -0.320       0.536
ma.L2         -0.6409      0.120     -5.349      0.000      -0.876      -0.406
ma.L3         -0.4194      0.248     -1.688      0.091      -0.906       0.068
sigma2         4.1332      0.032    129.631      0.000       4.071       4.196
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):            621185.23
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):            1134.96   Skew:                            11.07
Prob(H) (two-sided):                  0.00   Kurtosis:                       132.87
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
stepwise_fit = auto_arima(data_df['humidity'],
                          suppress_warnings=True)

stepwise_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  860
Model:               SARIMAX(0, 1, 1)   Log Likelihood               -2287.038
Date:                Thu, 10 Aug 2023   AIC                           4578.075
Time:                        12:44:45   BIC                           4587.587
Sample:                             0   HQIC                          4581.717
                                - 860                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.9858      0.005   -211.353      0.000      -0.995      -0.977
sigma2        11.9755      0.083    144.964      0.000      11.814      12.137
===================================================================================
Ljung-Box (L1) (Q):                   1.15   Jarque-Bera (JB):            706346.49
Prob(Q):                              0.28   Prob(JB):                         0.00
Heteroskedasticity (H):              40.39   Skew:                            11.18
Prob(H) (two-sided):                  0.00   Kurtosis:                       141.69
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
print(data_df.shape)
train=data_df.iloc[:20]
test=data_df.iloc[20:]
print(train.shape,test.shape)
print(test.iloc[0],test.iloc[-1])

(860, 2)
(20, 2) (840, 2)
humidity       19.799999
temperature     1.500000
Name: 1970-01-01 00:04:00, dtype: float64 humidity       19.299999
temperature     1.500000
Name: 1970-01-01 00:02:11, dtype: float64


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
temp_model=ARIMA(train['temperature'],order=(5,1,3))
temp_model=temp_model.fit()
temp_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:            temperature   No. Observations:                   20
Model:                 ARIMA(5, 1, 3)   Log Likelihood                  12.854
Date:                Thu, 10 Aug 2023   AIC                             -7.709
Time:                        12:44:45   BIC                              0.791
Sample:                             0   HQIC                            -6.270
                                 - 20                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0683      0.622      0.110      0.913      -1.151       1.288
ar.L2          0.1717      0.425      0.404      0.686      -0.660       1.004
ar.L3         -0.6460      0.337     -1.916      0.055      -1.307       0.015
ar.L4         -0.3159      0.505     -0.626      0.531      -1.305       0.673
ar.L5          0.1265      0.580      0.218      0.827      -1.009       1.262
ma.L1         -1.4679      1.029     -1.426      0.154      -3.485       0.549
ma.L2          1.4567      3.724      0.391      0.696      -5.842       8.755
ma.L3         -0.7969      2.487     -0.320      0.749      -5.671       4.077
sigma2         0.0100      0.024      0.425      0.671      -0.036       0.056
===================================================================================
Ljung-Box (L1) (Q):                   2.11   Jarque-Bera (JB):                 0.35
Prob(Q):                              0.15   Prob(JB):                         0.84
Heteroskedasticity (H):               0.31   Skew:                             0.33
Prob(H) (two-sided):                  0.18   Kurtosis:                         2.96
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
hum_model=ARIMA(train['humidity'],order=(0,1,1))
hum_model=hum_model.fit()
hum_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:               humidity   No. Observations:                   20
Model:                 ARIMA(0, 1, 1)   Log Likelihood                 -23.570
Date:                Thu, 10 Aug 2023   AIC                             51.140
Time:                        12:44:45   BIC                             53.029
Sample:                             0   HQIC                            51.459
                                 - 20                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.9985     12.970     -0.077      0.939     -26.420      24.423
sigma2         0.5994      7.752      0.077      0.938     -14.594      15.793
===================================================================================
Ljung-Box (L1) (Q):                   1.42   Jarque-Bera (JB):                 1.45
Prob(Q):                              0.23   Prob(JB):                         0.48
Heteroskedasticity (H):               0.86   Skew:                            -0.29
Prob(H) (two-sided):                  0.86   Kurtosis:                         1.78
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
tempmodel2=ARIMA(data_df['temperature'],order=(5,1,3))
tempmodel2=tempmodel2.fit()
data_df.tail()

,humidity,temperature
timestamp,,
1970-01-01 00:04:04,19.799999,1.5
1970-01-01 00:06:21,20.299999,1.7
1970-01-01 00:14:15,21.600000,1.5
1970-01-01 00:08:06,20.700001,1.6
1970-01-01 00:02:11,19.299999,1.5


In [ ]:
humodel2=ARIMA(data_df['humidity'],order=(0,1,1))
humodel2=humodel2.fit()
data_df.tail()

,humidity,temperature
timestamp,,
1970-01-01 00:04:04,19.799999,1.5
1970-01-01 00:06:21,20.299999,1.7
1970-01-01 00:14:15,21.600000,1.5
1970-01-01 00:08:06,20.700001,1.6
1970-01-01 00:02:11,19.299999,1.5


In [ ]:
temp_pred=tempmodel2.predict(start=len(data_df),end=len(data_df)+5,typ='levels').rename('ARIMA Predictions')

In [ ]:
print(temp_pred)

860    2.889772
861    2.882844
862    3.005230
863    2.937783
864    2.888571
865    2.769633
Name: ARIMA Predictions, dtype: float64


In [ ]:
hum_pred=humodel2.predict(start=len(data_df),end=len(data_df)+5,typ='levels').rename('ARIMA Predictions')

In [ ]:
print(hum_pred)

860    22.040834
861    22.040834
862    22.040834
863    22.040834
864    22.040834
865    22.040834
Name: ARIMA Predictions, dtype: float64
